In [2]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta

# my script
from w210_attribute_library_scale import tilekey, haversine_distance, getwf_final, weatherFeaturesSU, date_string

print("Date of reference: ", date_string)

Date of reference:  2022/06/01


In [3]:
datdir = "../data/"
attrs = "../attrs/"
modeld = "../model/"

# IMPORTANT - SET THE DATA OF REFERENCE
# date_string = '2022/06/01'

fweather = 'weather_data.csv'
dfw = pd.read_csv(datdir+fweather)
print(len(dfw))
dfw.head(1)

291511


,stn_wban,year,mo,da,min,max,temp,prcp,flag_prcp,rain_drizzle,...,usaf_wban,wban,lon,lat,elev,name,begin,end,state,country
0,998199_99999,2016,8,12,77.9,84.9,81.5,0.0,I,0,...,998199_99999,99999,-85.667,30.15,5.0,PANAMA CITY,20080721,20210920,FL,US


In [4]:
colweather = ['stn_wban', 'year', 'mo', 'da', 'temp', 'prcp', 'flag_prcp', 'min', 'max',
       'flag_prcp', 'rain_drizzle', 'sndp', 'fog', 'snow_ice_pellets', 'hail',
       'thunder', 'tornado_funnel_cloud', 'wdsp', 'mxpsd', 'gust', 'visib',
       'dewp', 'slp', 'usaf_wban', 'wban']
dfw1 = dfw[colweather]
dfw1.head(1)

,stn_wban,year,mo,da,temp,prcp,flag_prcp,min,max,flag_prcp,...,thunder,tornado_funnel_cloud,wdsp,mxpsd,gust,visib,dewp,slp,usaf_wban,wban
0,998199_99999,2016,8,12,81.5,0.0,I,77.9,84.9,I,...,0,0,10.4,17.1,999.9,999.9,9999.9,1015.3,998199_99999,99999


In [5]:
# Create a Date Field to be able to make date comparisons
dfw1["DateD"] = dfw1.apply(lambda row: datetime.strptime(str(row["year"])+"-"+str(row["mo"])+"-"+str(row["da"]),"%Y-%m-%d") ,axis=1)
dfw1.head(2)

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_12410/3998758606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfw1["DateD"] = dfw1.apply(lambda row: datetime.strptime(str(row["year"])+"-"+str(row["mo"])+"-"+str(row["da"]),"%Y-%m-%d") ,axis=1)


,stn_wban,year,mo,da,temp,prcp,flag_prcp,min,max,flag_prcp,...,tornado_funnel_cloud,wdsp,mxpsd,gust,visib,dewp,slp,usaf_wban,wban,DateD
0,998199_99999,2016,8,12,81.5,0.0,I,77.9,84.9,I,...,0,10.4,17.1,999.9,999.9,9999.9,1015.3,998199_99999,99999,2016-08-12
1,998199_99999,2016,11,22,57.2,0.0,I,47.8,66.6,I,...,0,4.5,8.0,999.9,999.9,9999.9,1022.4,998199_99999,99999,2016-11-22


In [6]:
# Replace values with NaN
no_valid = [99.99000, 999.900000, 9999.900000, 99999.000000]
dfw1 = dfw1.replace(99.99000,np.nan)
dfw1 = dfw1.replace(999.9000000,np.nan)
dfw1 = dfw1.replace(9999.900000,np.nan)
dfw1 = dfw1.replace(99999.000000,np.nan)

In [7]:
dfw1["DateD"].max(), dfw1["DateD"].min()

(Timestamp('2022-06-23 00:00:00'), Timestamp('2015-01-01 00:00:00'))

### Get Weather Features for Large Tiles

#### Using Timedelta=0 

**Note:** For model application, we only need timedelta zero

In [8]:
# Read file with large tiles
dfF2 = pd.read_csv(datdir+ "scale_up_F_tiles_ws_pair.csv")
dfF2.head(1)

,Unnamed: 0,name_x,imagenum,label,lon,lat,start_date,geometry,AnnualCrop,Forest,...,prediction,prediction_name,county_fp,Key_x,County,Key_y,name_y,lon_w,lat_w,Distance
0,174,0-2022-05-06-2022-07-05-200.00-0.jpeg_4_34.jpeg,0,0,-87.624327,30.880381,2022-05-06,POLYGON ((-87.62718590224753 30.87752176645508...,0.003029,0.009086,...,9,SeaLake,33,033_174,Escambia,720383_53847,WHITING FIELD NAVAL AIR STATI,-87.023,30.704,37.756111


In [9]:
daysdelta = timedelta(0)
is_scale = True
fields = ["Key_x", "Key_y", "name_y", "lon", "lat", "lon_w", "lat_w", "Distance"]
dfwf_large_0 = weatherFeaturesSU(dfF2, dfw1, is_scale, daysdelta, date_string, fields)
dfwf_large_0.head(1)

,Key,Key_y,date_ws,name,lon_t,lat_t,lon_w,lat_w,Distance,rolling_7_precip,...,y2_mean_prc,y2_max_prc,y2_mean_tmp,y2_max_tmp,y2_min_tmp,y3_mean_prc,y3_max_prc,y3_mean_tmp,y3_max_tmp,y3_min_tmp
0,033_174,720383_53847,2021-06-01,WHITING FIELD NAVAL AIR STATI,-87.624327,30.880381,-87.023,30.704,37.756111,0.63,...,0.093381,2.82,69.399178,85.9,35.5,0.173123,3.03,67.569589,85.3,37.6


In [10]:
dfwf_large_0.to_csv(attrs + "scale_up_F_tiles_ws_attr.csv", index=False)